# Example notebook for the _MOSES Sternfahrt 4_

This notebook downloads data from the MOSES Sternfahrt 4 mission using the restAPI and gets the model parameters of the ECOSMO model via the `ecosmo.api` module.

## The MOSES 4 Show Case

We select some sensors from the MOSES 4 Sternfahrt. These have been taken from the O2A Data Web Services at https://dashboard.awi.de/data-xxl

In [ ]:
sensors = [
    "small_scale_facility:pfb_awi_751801:longitude_0001",
    "small_scale_facility:pfb_awi_751801:latitude_0001",
    "small_scale_facility:pfb_awi_751801:temperature_0001",
    "small_scale_facility:pfb_awi_751801:temperature_sbe45_0001",
    "small_scale_facility:pfb_awi_751801:salinity_sbe45_0001",
]

## The MOSES 4 Show Case

To download the data, we use the O2A data-web-services package (https://github.com/o2a-data/o2a-data-dws).

In [ ]:
from dws import dws

df = dws.get(sensors, "2020-01-01", "2020-12-31")

df.columns = [col.split(":")[-1].replace("_0001", "").replace(" (mean) ", " ") for col in df.columns]
df

## The ECOSMO Backend Module

We created a backend module that loads climate model data and extracts the data along a specific path.

```python
def get_model_parameter(
    names: Union[Parameter, List[Parameter]],
    time: List[datetime.datetime],
    lat: List[float],
    lon: List[float],
) -> DataFrame:
    """Get ECOSMO model parameters for a given 3D path.

    This function takes a certain list of parameters and a 3D path, denoted by
    time, latitude and longitude, and extracts the data from the highresolution
    model output.

    Parameters
    ----------
    names : Union[Parameter, List[Parameter]]
        The parameter names to extract, see :class:`Parameter`.
    time : List[datetime.datetime]
        The list of times for each point.
    lat : List[float]
        The list of latitudes in degrees North for each point. Must be of the
        same length as `time` and `lon`.
    lon : List[float]
        The list of longitudes in degrees East for each point. Must be of the
        same length as `time` and `lat`.

    Returns
    -------
    DataFrame
        The dataframe with the selected model parameters for the given path.
    """
    if isinstance(names, str):
        names = [names]
    params = [param.value for param in map(Parameter, names)]

    if len(lat) != len(lon) or len(lat) != len(time):
        raise ValueError("lat, time and lon must all be of the same length!")

    with xr.open_dataset(
        osp.join(data_dir, "ecosmo-ute-daewel-20200501-20200531.nc")
    ) as ds:

        ds = ds[params].isel(layer=0).drop_vars("layer")
        lon_da = xr.DataArray(lon, dims="path")
        lat_da = xr.DataArray(lat, dims="path")
        time_da = xr.DataArray(time, dims="path")
        ds = ds.interp(lon=lon_da, lat=lat_da, time=time_da)

    # convert the data to a pandas dataframe
    return ds.to_dataframe()
```

## Using the ECOSMO api module

Now we call the backend module (assuming that it is connected already).

In [ ]:
from ecosmo.api import get_model_parameter

model_data = get_model_parameter(
    ["temp", "salt"], 
    df.datetime.to_list(), 
    df["latitude [degree]"].to_list(), 
    df["longitude [degree]"].to_list()
)

In [ ]:
model_data

## Model vs. Observations

Now we can merge the `temp` and `salt` columns into our observations and plot them.

In [ ]:
df["ecosmo_temperature"] = model_data.temp.values
df["ecosmo_salinity"] = model_data.salt.values

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 3, figsize=(14, 4), dpi=150)
df.plot.scatter("temperature [°C]", "ecosmo_temperature", ax=axes[0])
df.plot.scatter("temperature_sbe45 [°C]", "ecosmo_temperature", ax=axes[1])
df.plot.scatter("salinity_sbe45 [PSU]", "ecosmo_salinity", ax=axes[2])